### 0~10 라벨까지 1초 단위로 제스처 랜드마크 수집 (test dataset)
- 잘못 수집되면 해당 라벨 랜드마크를 다시 수집할 수 있도록 수정
- space bar : 수집 시작
- s : 저장
- r : 재시도

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time

# 저장 경로
save_dir = './test_data_2/'
os.makedirs(save_dir, exist_ok=True)

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 입력
VIDEO_URL = 'http://192.168.0.103:5000/video'
cap = cv2.VideoCapture(VIDEO_URL)

# 제스처 라벨
gesture_label = {
    0: 'Right',
    1: 'Left',
    2: 'Turn Clockwise',
    3: 'Turn Anticlockwise',
    4: 'Twinkle',
    5: 'Okay',
    6: 'Stop',
    7: 'Rock',
    8: 'Play',
    9: 'Slide',
    10: 'None'
}

max_label = max(gesture_label.keys())
current_label = 0
seq_num = 1
window_size = 30

# 직원별로 수집 ID 관리
id = '10'

print("▶ Space: 수집 시작 | s: 저장 | r: 다시 수집 | q: 종료")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    display_frame = frame.copy()

    label_name = gesture_label[current_label]
    cv2.putText(display_frame, f"Label {current_label} - {label_name}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(display_frame, f"Press SPACE to record", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.imshow('Collector', display_frame)

    key = cv2.waitKey(1)
    if key == ord('q') or current_label > max_label:
        print("수집 종료.")
        break

    elif key == ord(' '):
        print(f"▶ Label {current_label} - {label_name} 수집 시작 (1초)...")
        frame_buffer = []
        start_time = time.time()

        while time.time() - start_time < 1.0:
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(rgb)

            if result.multi_hand_landmarks:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # 벡터 및 각도 계산
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :]
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :]
                    v = v2 - v1
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18], :],
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19], :]))
                    angle = np.degrees(angle)

                    # concat: joint + angle + label
                    angle = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle, current_label)
                    joint_flat = joint.flatten()
                    data = np.concatenate([joint_flat, angle_label])
                    frame_buffer.append(data)

            mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS) if result.multi_hand_landmarks else None
            cv2.putText(frame, f"Recording: {label_name}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Collector', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        print("⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]")
        while True:
            confirm_key = cv2.waitKey(0)
            if confirm_key == ord('s'):
                filename = f"data_gesture_{label_name}_{id}.csv"
                np.savetxt(os.path.join(save_dir, filename), np.array(frame_buffer), delimiter=",")
                print(f"저장 완료: {filename}")
                seq_num += 1
                current_label += 1
                # 라벨 수집 완료 후 자동 종료
                if current_label > max_label:
                    print("모든 제스처 수집 완료!")
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()
                break
            elif confirm_key == ord('r'):
                print(f"↩ 다시 수집: Label {current_label} - {label_name}")
                break
            elif confirm_key == ord('q'):
                print("사용자 종료")
                cap.release()
                cv2.destroyAllWindows()
                exit()

cap.release()
cv2.destroyAllWindows()


2025-06-19 12:16:14.690513: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-19 12:16:14.692059: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-19 12:16:14.717531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-19 12:16:14.717566: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-19 12:16:14.718336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

▶ Space: 수집 시작 | s: 저장 | r: 다시 수집 | q: 종료
▶ Label 0 - Right 수집 시작 (1초)...


W0000 00:00:1750302983.908006 1007738 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Right_10.csv
▶ Label 1 - Left 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Left_10.csv
▶ Label 2 - Turn Clockwise 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Turn Clockwise_10.csv
▶ Label 3 - Turn Anticlockwise 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Turn Anticlockwise_10.csv
▶ Label 4 - Twinkle 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Twinkle_10.csv
▶ Label 5 - Okay 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Okay_10.csv
▶ Label 6 - Stop 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Stop_10.csv
▶ Label 7 - Rock 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Rock_10.csv
▶ Label 8 - Play 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Play_10.csv
▶ Label 9 - Slide 수집 시작 (1초)...
⏸ 수집 종료. [s: 저장 / r: 다시 / q: 종료]
저장 완료: data_gesture_Slide_10.csv
▶ Label 10 - N

: 

### 30fps와 15fps 비교 

In [7]:
import cv2
import os
import time

save_dir_30 = './frames_30fps'
save_dir_15 = './frames_15fps'
os.makedirs(save_dir_30, exist_ok=True)
os.makedirs(save_dir_15, exist_ok=True)

# 웹캠 또는 스트리밍 주소
VIDEO_URL = 'http://192.168.0.103:5000/video'
cap = cv2.VideoCapture(VIDEO_URL)

# 녹화 시작
print("▶ Space 키를 누르면 1초 프레임 수집 시작")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    cv2.putText(frame, "Press SPACE to start", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Preview', frame)

    key = cv2.waitKey(1)
    if key == ord(' '):
        print("▶ 수집 시작 (1초)...")
        collected_frames = []
        start = time.time()
        while time.time() - start < 1.0:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            collected_frames.append(frame.copy())
            cv2.imshow('Recording', frame)
            cv2.waitKey(1)
        break
    elif key == ord('q'):
        break

# 저장
for i, f in enumerate(collected_frames):
    cv2.imwrite(os.path.join(save_dir_30, f"frame_{i:02d}.jpg"), f)

# 15fps는 2프레임에 하나씩 저장
for i in range(0, len(collected_frames), 2):
    cv2.imwrite(os.path.join(save_dir_15, f"frame_{i//2:02d}.jpg"), collected_frames[i])

print(f"✅ 총 {len(collected_frames)} 프레임 수집 완료 (30fps 기준)")
print("✅ 30fps와 15fps 프레임 이미지 저장 완료")
cap.release()
cv2.destroyAllWindows()


▶ Space 키를 누르면 1초 프레임 수집 시작
▶ 수집 시작 (1초)...
✅ 총 31 프레임 수집 완료 (30fps 기준)
✅ 30fps와 15fps 프레임 이미지 저장 완료
